In [ ]:
! pip install -qq -U langchain-openai langchain-core langchain

In [31]:
import os
import openai
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import  RunnableLambda, RunnableBranch
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
chat_model = ChatOpenAI(
    model_name = "gpt-4o-mini",
    temperature = 0.6
)

In [ ]:
input_messages = [
    "what is the capital of Japan?", #question - answer
    "Once upon a time in a faraway land, there was a brave knight who fought dragons to protect his kingdom.", #story - summarize
    "Technology is evolving rapidly, and AI is shaping the future of work . ", #statement - generate title
    "self-driving cars will change the way we travel." #statement - generate title
]

In [19]:
classification_prompt  = ChatPromptTemplate.from_messages([
    SystemMessage(content = "You are an AI model. Your work is to classify whether the provided input text is either question or statement or story and return the category only"),
    ("human", "Classify the text :\n\n {text} \n category: ")
])

In [20]:
classification_step = classification_prompt | chat_model | (lambda x:{"category":x.content.strip().lower()})

In [21]:
answer_prompt  = ChatPromptTemplate.from_messages([
    SystemMessage(content = "You are an AI model. Your work is to answer the question accurately and reply with I don't know if you aren't aware of answer"),
    ("human", "Answer the question :\n\n {text} ")
])
summary_prompt  = ChatPromptTemplate.from_messages([
    SystemMessage(content = "You are an AI model. Your work is to summarize the provided text"),
    ("human", "summarize the text :\n\n {text}")
])
title_prompt  = ChatPromptTemplate.from_messages([
    SystemMessage(content = "You are an AI model. Your work is to provide the suitable title for the statement provided."),
    ("human", "provide the title that aptly suits for the following text :\n\n {text}")
])

In [22]:
answer_question = answer_prompt | chat_model | (lambda x : {"response" : x.content.strip()})
summarize_story = summary_prompt | chat_model | (lambda x : {"response" : x.content.strip()})
generate_title = title_prompt | chat_model | (lambda x : {"response" : x.content.strip()})

In [29]:
pipeline = (
    RunnableLambda(lambda x : {"text" : x}) | 
    (lambda x : {**x, **classification_step.invoke(x)}) |
    RunnableBranch(
        (lambda x : x["category"] == "question", answer_question),
        (lambda x : x["category"] == "story", summarize_story),
        (generate_title)
    )
)

In [30]:
for message in input_messages:
    response = pipeline.invoke(message)
    print(response)

{'response': 'The capital of Japan is Tokyo.'}
{'response': 'In a distant land, a courageous knight battled dragons to safeguard his kingdom.'}
{'response': '"Embracing the Future: The Impact of AI on the Evolving Workplace"'}
{'response': '"Revolutionizing Travel: The Impact of Self-Driving Cars"'}
